<a href="https://colab.research.google.com/github/poorvaja-sathasivam/Emotion-Detection-/blob/main/EmotionDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Importing all the necessary libraries. 
import tensorflow as tf 
import os 
import numpy as np 
import matplotlib.pyplot as plt 
import cv2 

In [ ]:
# from google.colab import files
# uploaded = files.upload()

In [ ]:
!unzip -q Data.zip

unzip:  cannot find or open Data.zip, Data.zip.zip or Data.zip.ZIP.


In [ ]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'
# Getting the images for training 
imgArr = cv2.imread("Data/Training/1/Training_3908.jpg")

In [ ]:
# Checking size of the image 
imgArr.shape

AttributeError: ignored

In [ ]:
# Displaying the image
plt.imshow(imgArr)

In [ ]:
# Creating a data directory 
DirectoryOfData = "Data/Training/" 

In [ ]:
# Creating a list of classes which are named the same as the folder
Classes = ["1", "2", "3", "4", "5", "6", "7"]

In [ ]:
# Reading all the images in the folder 
for Category in Classes:
    path = os.path.join(DirectoryOfData, Category)
    for img in os.listdir(path):
        imgArr = cv2.imread(os.path.join(path,img))
#         imgtorgb = cv2.cvtColor(imgArr,cv2.COLOR_GRAY2RGB)
        plt.imshow(cv2.cvtColor(imgArr, cv2.COLOR_BGR2RGB))
        plt.show()
        break 
    break

In [ ]:
# Changing the size of the image 
# As ImageNet has a size of 224 x 224, the size is set as 224 
sizeOfImage = 224 
newArr = cv2.resize(imgArr,(sizeOfImage,sizeOfImage))
plt.imshow(cv2.cvtColor(newArr,cv2.COLOR_BGR2RGB))
plt.show()
# Therefore, the size will change from 48x48 to 224x224 

In [ ]:
# Checking if the size has changed 
newArr.shape

In [ ]:
# Reading all the images and converting them to an Array 
trainingSet = []

# Function to create the training data 
def createTrainingSet():
    for Category in Classes:
        path = os.path.join(DirectoryOfData, Category)
        numOfClass = Classes.index(Category)
        for img in os.listdir(path):
            try:
                imgArr = cv2.imread(os.path.join(path,img))
                newArr = cv2.resize(imgArr, (sizeOfImage, sizeOfImage))
                trainingSet.append([newArr,numOfClass])
            except Expection as e:
                pass

In [ ]:
# Calling the function 
createTrainingSet()

In [ ]:
# Checking the number of images that has read by the function 
print(len(trainingSet))

In [ ]:
# To avoid making the model learn the sequence, shuffling of the data is used 
import random 
random.shuffle(trainingSet)
# This helps the model to be more dynamic and robust 

In [ ]:
# A and B are variables used for features and label  
A = [] 
B = []

for feature, label, in trainingSet:
    A.append(feature)
    B.append(label)

# Converting to 4 dimensions. 
A = np.array(A).reshape(-1,sizeOfImage, sizeOfImage, 3)

# Checking the shape 
A.shape

In [ ]:
# # Normalising the data where 255 stands for black(maximum) and 0 stands for white(minimum)
# A = A/255.0; 

In [ ]:
# Checking the type of B 
type(B)

In [ ]:
# Converting to an array 
BB = np.array(B) 
# Checking the shape 
BB.shape

In [ ]:
# Importing the libraries 
import tensorflow as tf 
from tensorflow import keras 
from tensorflow.keras import layers

In [ ]:
# Pre-trained model 
model = tf.keras.applications.MobileNetV2()

In [ ]:
model.summary()

In [ ]:
# The model is trained for 1000 classes which is showed as predictions(Dense) : (None, 1000) ==> therefore performing transfer learning 
baseInput = model.layers[0].input

In [ ]:
# Getting only 7 classes 
baseOutput = model.layers[-2].output

In [ ]:
baseOutput

In [ ]:
# 1. Creating a new layer after the global average pooling layer (None, 1280)
finOutput = layers.Dense(128)(baseOutput)
# 2. Creating the activation function 
fOutput = layers.Activation('relu')(finOutput)
# 3. Creating another layer 
finOutput = layers.Dense(64)(finOutput)
# 4. Creating the activation function 
fOutput = layers.Activation('relu')(finOutput)
# Since the classes are 7, it will be reduced from 1000 to 7 
finOutput = layers.Dense(7,activation = 'softmax')(finOutput)

In [ ]:
finOutput

In [ ]:
newModel = keras.Model(inputs = baseInput, outputs = finOutput)

In [ ]:
newModel.summary()

In [ ]:
newModel.compile(loss="sparse_categorical_crossentropy", optimizer ="adam", metrics =["accuracy"])

In [ ]:
newModel.fit(A,BB, epochs = 1)

In [ ]:
newModel.save('Model.h5')

In [ ]:
newModel = tf.keras.models.load_model("Model.h5")

In [ ]:
newModel.evaluate 

In [ ]:
new = cv2.imread("girl.jpg")

In [ ]:
new.shape

In [ ]:
plt.imshow(cv2.cvtColor(new,cv2.COLOR_BGR2RGB))